In [20]:
import pandas as pd
import numpy as np

bible_df = pd.read_csv('D:/LO_resources/eng-BSB.csv', encoding='utf-8', index_col=0)
bible_df = bible_df.astype({'verse_number':int,'chapter_number':int})
references_df = pd.read_csv('D:/LO_resources/prayer_ref_quoted.csv', header=None)
references_df.rename(columns={0: 'verse'}, inplace=True)
mention_df = pd.read_csv('D:/LO_resources/prayer_ref_mention.csv', header=None)
mention_df.rename(columns={0: 'verse'}, inplace=True)
answered_df = pd.read_csv('D:/LO_resources/prayer_ref_answered.csv', header=None)
answered_df.rename(columns={0: 'verse'}, inplace=True)

bible_df

,verse,text,book,verse_number,chapter_number,verse_length,verse_word_count,traditional_group,traditional_book_order,alternative_group,alternative_book_order
2,Genesis 1:1,In the beginning God created the heavens and t...,Genesis,1,1,55.0,10.0,Law,1,Covenant History,1
3,Genesis 1:2,"Now the earth was formless and void, and darkn...",Genesis,2,1,150.0,29.0,Law,1,Covenant History,1
4,Genesis 1:3,"And God said, “Let there be light,” and there ...",Genesis,3,1,56.0,11.0,Law,1,Covenant History,1
5,Genesis 1:4,"And God saw that the light was good, and He se...",Genesis,4,1,82.0,16.0,Law,1,Covenant History,1
6,Genesis 1:5,"God called the light “day,” and the darkness H...",Genesis,5,1,123.0,21.0,Law,1,Covenant History,1
...,...,...,...,...,...,...,...,...,...,...,...
31099,Revelation 22:17,"The Spirit and the bride say, “Come!” Let the ...",Revelation,17,22,165.0,33.0,Letter,66,New Testament,66
31100,Revelation 22:18,I testify to everyone who hears the words of p...,Revelation,18,22,147.0,29.0,Letter,66,New Testament,66
31101,Revelation 22:19,And if anyone takes away from the words of thi...,Revelation,19,22,167.0,34.0,Letter,66,New Testament,66
31102,Revelation 22:20,"He who testifies to these things says, “Yes, I...",Revelation,20,22,87.0,16.0,Letter,66,New Testament,66


In [21]:
BOOK_NAMES_LIST = ['Genesis', 'Exodus', 'Leviticus', 'Numbers', 'Deuteronomy', 'Joshua', 'Judges', 'Ruth', '1 Samuel', '2 Samuel', '1 Kings', '2 Kings', '1 Chronicles', '2 Chronicles', 'Ezra', 'Nehemiah', 'Esther', 'Job', 'Psalm', 'Proverbs', 'Ecclesiastes', 'Song of Solomon', 'Lamentations', 'Isaiah', 'Jeremiah', 'Ezekiel', 'Daniel', 'Hosea', 'Joel', 'Amos', 'Obadiah', 'Jonah', 'Micah', 'Nahum', 'Habakkuk', 'Zephaniah', 'Haggai', 'Zechariah', 'Malachi', 'Matthew', 'Mark', 'Luke', 'John', 'Acts', 'Romans', '1 Corinthians', '2 Corinthians', 'Galatians', 'Ephesians', 'Philippians', 'Colossians', '1 Thessalonians', '2 Thessalonians', '1 Timothy', '2 Timothy', 'Titus', 'Philemon', 'Hebrews', 'James', '1 Peter', '2 Peter', '1 John', '2 John', '3 John', 'Jude', 'Revelation']
BOOK_ABBREV_LIST_2 = []
BOOK_ABBREV_LIST_3 = []
BOOK_ABBREV_LIST_4 = []
for book_name in BOOK_NAMES_LIST:
    if book_name[0].isdigit():
        BOOK_ABBREV_LIST_4.append(book_name[:6])  # Keep the number and the first 4 letters
        BOOK_ABBREV_LIST_3.append(book_name[:5])  # Keep the number and the first 3 letters
        BOOK_ABBREV_LIST_2.append(book_name[:4])
    else:
        BOOK_ABBREV_LIST_4.append(book_name[:4])  # Print the first 4 letters
        BOOK_ABBREV_LIST_3.append(book_name[:3])  # Print the first 3 letters
        BOOK_ABBREV_LIST_2.append(book_name[:2])

# Create a dictionary to map the short names to their full names
book_name_dict = dict(zip(BOOK_ABBREV_LIST_4 + BOOK_ABBREV_LIST_3 + BOOK_ABBREV_LIST_2, BOOK_NAMES_LIST * 3))
combined_list = BOOK_NAMES_LIST+BOOK_ABBREV_LIST_4+BOOK_ABBREV_LIST_3+BOOK_ABBREV_LIST_2

In [22]:
def add_columns_from_verse_references(df):
    # replace – with -
    df['verse'] = df['verse'].replace('–', '-', regex=True)
    # get details
    df['short_name'] = df['verse'].str.extract(f"({'|'.join(combined_list)})")
    df['verse_number'] = df['verse'].str.split(':').str.get(-1)
    df['chapter_number'] = df['verse'].str.split(':').str.get(0).str.split(' ').str.get(-1)
    # correct for if no verse numbers are given
    
    #add book
    df['book'] = df['short_name'].map(book_name_dict)
    # if no match assume short_name was book name
    df['book'] = df['book'].fillna(df['short_name'])
    
    # check all matches found
    print(len(df),len(df[df['book'].isnull()]))
    return df

In [23]:
references_df = add_columns_from_verse_references(references_df) #268,0

268 0


In [24]:
mention_df = add_columns_from_verse_references(mention_df)

83 0


In [25]:
answered_df = add_columns_from_verse_references(answered_df)

119 0


In [26]:
def label_bible_with_matches(df,label='None'):

    combined_mask = (bible_df['book'] == 'None')
    # Define the reference you want to match
    for i in range(0,len(df)):
        book_name = df['book'].iloc[i]
        chapter_number = df['chapter_number'].iloc[i]
        verse_number = df['verse_number'].iloc[i]
        verse = df['verse'].iloc[i]
        if '-' in verse_number:
            start_verse = verse_number.split('-')[0]
            end_verse = verse_number.split('-')[1]
        else:
            start_verse = end_verse = verse_number
    
        # Create a boolean mask to filter the rows
        if ':' in verse:
            mask = (
                (bible_df['book'] == book_name) &
                (bible_df['chapter_number']== int(chapter_number)) &
                (bible_df['verse_number'].between(int(start_verse), int(end_verse)))
            )
        else:
             mask = (
                (bible_df['book'] == book_name) &
                (bible_df['chapter_number']== int(chapter_number))
            )
        combined_mask = np.logical_or(combined_mask, mask)
    
    # Filter the DataFrame using the mask
    bible_df['match_verse'] = combined_mask
    bible_df.to_csv(f'D:/LO_resources/bible_with_{label}_matches.csv', encoding='utf-8')
    bible_df[bible_df['match_verse']]['text'].to_csv(f'D:/LO_resources/bible_{label}_matches_only.csv', encoding='utf-8')

In [27]:
label_bible_with_matches(references_df,label='reference')
label_bible_with_matches(mention_df,label='mention')
label_bible_with_matches(answered_df,label='answer')